# Webmining - Assignment 1

This **Home Assignment** is to be submitted and you will be given points for each of the tasks. It familiarizes you with basics of *web crawling* and standard text preprocessing. It then takes a deep dive into *GloVe* one approach for obtaining word embeddings. To train GloVe we will frist construct the co-occurence matrix and then we will use adaptive stochastic gradient descent to minimize the cost function.

## Formalities
**Submit in a group of 2-3 people until 27.05.2020 23:59CET. The deadline is strict!**

## Evaluation and Grading
General advice for programming excercises at *CSSH*:
Evaluation of your submission is done semi automatically. Think of it as this notebook being 
executed once. Afterwards, some test functions are appended to this file and executed respectively.

Therefore:
* Submit valid _Python3_ code only!
* Use external libraries only when specified by task.
* Ensure your definitions (functions, classes, methods, variables) follow the specification if
  given. The concrete signature of e.g. a function usually can be inferred from task description, 
  code skeletons and test cases.
* Ensure the notebook does not rely on current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
  * Double check if your code relies on presence of files or directories other than those mentioned
    in given tasks. Tests run under Linux, hence don't use Windows style paths 
    (`some\path`, `C:\another\path`). Also, use paths only that are relative to and within your
    working directory (OK: `some/path`, `./some/path`; NOT OK: `/home/alice/python`, 
    `../../python`).
* Keep your code idempotent! Running it or parts of it multiple times must not yield different
  results. Minimize usage of global variables.
* Ensure your code / notebook terminates in reasonable time.

**There's a story behind each of these points! Don't expect us to fix your stuff!**

Regarding the scores, you will get no points for a task if:
- your function throws an unexpected error (e.g. takes the wrong number of arguments)
- gets stuck in an infinite loop
- takes much much longer than expected (e.g. >1s to compute the mean of two numbers)
- does not produce the desired output (e.g. returns an descendingly sorted list even though we asked for ascending, returns the mean and the std even though we asked for only the mean, prints an output instead of returning it, ...)

In [241]:
# credentials of all team members (you may add or remove items from the dictionary)
team_members = [
    {
        'first_name': 'Pavel',
        'last_name': 'Raschetnov',
        'student_id': 404839
    },
    {
        'first_name': 'Anya',
        'last_name': 'Poudyal',
        'student_id': 391805
    },
    {
        'first_name': 'Philipp',
        'last_name': 'Stein',
        'student_id': 397615
    }
]

## 1. Crawling web pages (total of 4 points)
Consider the top 150 stackoverflow questions tagged with data-mining ordered by votes (e.g. the  questions from the 10 first pages accessible from here: https://stackoverflow.com/questions/tagged/data-mining?tab=votes&pagesize=15). Use the `BeautifulSoup`  and `requests` package.

### a) Simple spidering (1.5)

Write a function ```get_questions``` that takes a tag (like ```"data-mining"```) and a number `n` and returns a list containing the hyperlinks (as strings) to the top n questions as explained above. Assume the tag exists.

(the first link for data-mining is: https://stackoverflow.com/questions/12146914/what-is-the-difference-between-linear-regression-and-logistic-regression)

### b) Processing a page (1.5)

Write a function ```process_question``` that takes a string hyperlink to a stackoverflow questions and returns a dictionary. It contains the text of the question, their comments and the answers to the question and comments. Keep in mind to remove: All images, tags, html tags, user_information, _code_ sections. Also remove information on edits, dates etc. Finally remove all functional text-content like share, edit, follow, flag, add comment (the kind of button things). Remove everything that is not inside the div with `id="mainbar"`.

The structure of the result is:

```python
{'title': 'The title',
 'question': {'text' : 'How to learn web-mining?',
              'comments':['Good question', 'Sounds\n interesting']},
 'answers' : [{'text':'Do a course at CSSH!', 
               'comments' : ['You will learn a lot', 'Good stuff']},
              {'text':'Learn on youtube', 'comments' : []}, ]}
```
You can also find an example of the  processed page https://stackoverflow.com/questions/12146914/what-is-the-difference-between-linear-regression-and-logistic-regression on moodle.

### c) Flatten the document (0.5)

Write a function `flatten_question` that takes a dict of the structure from c) and returns a list of strings for that dict. Therefor merge the title with the flattened question. Then add the one string for each answer.
The answer and question are flattened by first joining the comment strings with a `" "` and then joining the text and the comments in the same way. The text should preceed the comments.
The returned list should look like:

```
['The title How to learn web-mining? Good question Sounds\n interesting', 'Do a course at CSSH! You will learn a lot Good stuff', 'Learn on youtube ']
```

### d) Bringing it all together (0.5)

Write a function `process_top_questions` that takes a tag and a number n and that processes the top n questions by votes as explained above. It returns a single list of strings (concatenated from the list of strings for each single answer). Thereby use the previously defined functions.

Execute the function with the tag `"data-mining"` and n=150. Store the result in ```result_1```


In [4]:
from bs4 import BeautifulSoup
import requests

In [226]:
def get_questions(tag, n):
    urls = []
    page = 1
    while len(urls) < n:
        r = requests.get(f'https://stackoverflow.com/questions/tagged/{tag}?tab=votes&pagesize=50', 
                     params={'page': page})
        soup = BeautifulSoup(r.text)

        base_url = 'https://stackoverflow.com'
        found = False
        for a_tag in soup.select('div.question-summary h3 a'):
            found = True
            urls.append(base_url + a_tag['href'])
            
        if not found: 
            break
            
        page += 1
    return urls[:n]

In [237]:
def process_question(url):
    print('processing ' + str(url))
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    mainbar = soup.select_one('div#mainbar')
    
    result = {}
    
    result['title'] = soup.select_one('div#question-header h1 a').get_text()
    
    def parse_comments(el):
        comments = []
        for comment_el in el.select('div.comments div.comment-text'):
            comment = ' '.join(x.get_text() for x in comment_el.select('span.comment-copy'))
            comments.append(comment)
        return comments
    
    
    question_el = mainbar.select_one('div#question')
    question_text = ' '.join(x.get_text() for x in question_el.select('div.post-text p'))
    result['question'] = {
        'text': question_text,
        'comments': parse_comments(question_el)
    }
    
    result['answers'] = []
    for answer_el in mainbar.select('div#answers div.answer'):

        text = '\n'.join(p.get_text() for p in answer_el.select('div.post-text p'))

        answer = {
            'text': text,
            'comments': parse_comments(answer_el)
        }

        result['answers'].append(answer)
    
    return result   

In [228]:
# example input
d={'title': 'The title',
 'question': {'text' : 'How to learn web-mining?',
              'comments':['Good question', 'Sounds\n interesting']},
 'answers' : [{'text':'Do a course at CSSH!', 
               'comments' : ['You will learn a lot', 'Good stuff']},
              {'text':'Learn on youtube', 'comments' : []}, ]}

def flatten_question(d):
    def merge_text_comments(d):
        return d['text'] + ' ' + ' '.join(d['comments'])
    
    result =  [d['title'] + ' ' + merge_text_comments(d['question']),]
    for a in d['answers']:
        result.append(merge_text_comments(a))
    return result

In [229]:
def process_top_questions(tag, n):
    from itertools import chain
    
    flattened_dicts = [flatten_question(process_question(q)) for q in get_questions(tag, n)]
    return list(chain.from_iterable(flattened_dicts))

In [239]:
tag,n = 'data-mining', 150
result_1 = process_top_questions(tag, n)

processing https://stackoverflow.com/questions/12146914/what-is-the-difference-between-linear-regression-and-logistic-regression
processing https://stackoverflow.com/questions/1746501/can-someone-give-an-example-of-cosine-similarity-in-a-very-simple-graphical-wa
processing https://stackoverflow.com/questions/5064928/difference-between-classification-and-clustering-in-data-mining
processing https://stackoverflow.com/questions/2323768/how-does-the-amazon-recommendation-feature-work
processing https://stackoverflow.com/questions/17469835/why-does-one-hot-encoding-improve-machine-learning-performance
processing https://stackoverflow.com/questions/11808074/what-is-an-intuitive-explanation-of-the-expectation-maximization-technique
processing https://stackoverflow.com/questions/26355942/why-is-the-f-measure-a-harmonic-mean-and-not-an-arithmetic-mean-of-the-precision
processing https://stackoverflow.com/questions/11513484/1d-number-array-clustering
processing https://stackoverflow.com/question

processing https://stackoverflow.com/questions/12155068/an-understandable-clusterization
processing https://stackoverflow.com/questions/7076349/is-there-a-good-way-to-do-this-type-of-mining
processing https://stackoverflow.com/questions/2680853/architecture-for-database-analytics
processing https://stackoverflow.com/questions/15465997/python-tools-for-out-of-core-computation-data-mining
processing https://stackoverflow.com/questions/39480493/find-substring-in-text-which-has-the-highest-similarity-to-a-given-keyword
processing https://stackoverflow.com/questions/7643512/nlp-and-machine-learning-for-sentiment-analysis
processing https://stackoverflow.com/questions/6685961/weka-simple-k-means-clustering-assignments
processing https://stackoverflow.com/questions/25181104/cosine-distance-as-vector-distance-function-for-k-means
processing https://stackoverflow.com/questions/7059954/latent-semantic-analysis-concepts
processing https://stackoverflow.com/questions/46999771/comparing-a-large-num

# 2. Preprocessing (total of 2 points)
It is/was common to use stopword removal and stemming as a preparation for word_embeddings. You should use the stopword list provided by Python-nltk library.

## a) The usual preprocessing (1)

Write a function `to_tokens` that takes a string and performs tokenization, stopword removal and stemming. It returns a list of tokens. The tokens are in the same order they were in the initial string. Use the functions from the nltk library.
To transform a string into tokens use the function `nltk.word_tokenize`.

## b) Reduce vocabulary (1)
Write a function `process_and_filter_corpus` that takes a list of strings and an integer as input. It applies the `to_tokens` function on each of those. The return values are

1) the list of list of tokens. All tokens that appear less than min_support times (in the entire corpus) are __removed__

2) a set of tokens that were removed.



In [285]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import string
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/pavel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pavel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [302]:
def to_tokens(s):
    tokens = nltk.word_tokenize(s)
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if not all(c in string.punctuation for c in t)]

    tokens = [t for t in tokens if t not in stopwords.words('english')]

    stemmer = SnowballStemmer("english")
    tokens = [stemmer.stem(t) for t in tokens]
    
    return tokens

In [301]:
from collections import Counter
def process_and_filter_corpus(strings, min_support):
    counts = Counter()
    result = [to_tokens(s) for s in strings]
    for tokens in result:
        counts.update(tokens)
    removed_tokens = {k for k,v in counts.items() if v < min_support}
    
    filtered_result = []
    for tokens in result:
        tokens = [t for t in tokens if t not in removed_tokens]
        if len(tokens) > 0:
            filtered_result.append(tokens)
    
    return filtered_result, removed_tokens

## 3. Glove (total of 10 points)
### a) Computing the global word co-occurence matrix (3 = 2.5 + 0.5)
We will now explore some steps of required for the GloVe word embedding.

Write a function `get_cooc_matrix` that takes a list of list of tokens and an integer _context_size_ as input. It returns 1) a sparse co-occurrence matrix (a dict mapping pairs of integer indices to their cooc-score) and 2) a dictionary that maps a word to an index in the cooc matrix/dict. Do not use any non standard library for that. If two tokens are d places apart they get a score of 1/d. Only take into account words that are at most context_size apart from the central word.

Example:

For the corpus `[['bad', 'dog', 'bad', 'cat', 'thing'],['bad', 'dog']]` and context_size=2 the tokens 'dog' and 'thing' do not co-occur. While the tokens 'dog' and 'bad' have a cooc score of 3. The token pair 'bad' and 'bad' has a cooc score of 1.0.

Additionally write a function `cooc_to_numpy` that takes the dict-sparse-representation and returns three numpy arrays. The first two are of type int and they contain the values for i and j respectively. The third array contains the cooc-score for that entry. They are sorted in a way that num it is first sorted ascending according to i and then ascending according to j. This sorting is only for reproducability, not necessary for the glove. We call the output of this function a coord_tpl

In [345]:
import numpy as np
from collections import defaultdict

def get_cooc_matrix(tokens_list, context_size):
    vocab = {}
    for tokens in tokens_list:
        for t in tokens:
            if t not in vocab:
                vocab[t] = len(vocab)
    
    scores = defaultdict(float)
    
    for tokens in tokens_list:
        for i in range(len(tokens)):
            for d in range(1, context_size+1):
                j1 = i - d
                j2 = i + d
                t = vocab[tokens[i]]
                if j1 >= 0:
                    t1 = vocab[tokens[j1]]
                    scores[(t,t1)] += 1/d
                if j2 < len(tokens):
                    t2 = vocab[tokens[j2]]
                    scores[(t,t2)] += 1/d
    
    return scores, vocab
        
def cooc_to_numpy(coocs):
    i = []
    j = []
    s = []
    for k in sorted(coocs.keys()):
        i.append(k[0])
        j.append(k[1])
        s.append(coocs[k])
    return (np.array(i), np.array(j), np.array(s))

mini_corpus=[['bad', 'dog', 'bad', 'cat', 'thing'], ['bad', 'dog']]

#results for mini_corpus:
#d,vocab = get_cooc_matrix(mini_corpus, 2)
#cooc_tpl = cooc_to_numpy(d)
# print('cooc_tpl',(
#  np.array([0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3]),
#  np.array([0, 1, 2, 3, 0, 2, 0, 1, 3, 0, 2]),
#  np.array([1. , 3. , 1. , 0.5, 3. , 0.5, 1. , 0.5, 1. , 0.5, 1. ])))
# print('vocab', {'bad': 0, 'dog': 1, 'cat': 2, 'thing': 3})

Now that we have obtained the global co-occurence matrix, it is time to use this information to train GloVe vectors.
For this task we will use an (adaptive) stochastic gradient descent method.

### b) Initialize Vectors and Gradients (1)
Write a function `init_matrix`  that takes three arguments: 1) the size of the vocabulary $v$ 2) the size of the desired Glove vectors $n$ and 3) a random number generator which is initialized like: (https://numpy.org/doc/1.18/reference/random/index.html). It returns two matrices: 1) The matrix of glove vectors where each row is of size $n+1$ reflecting one glove vector + bias. It is initialized with random values such that they lie uniformly on the interval $[-l, l)$ with $l=\frac{0.5}{n}$. Use one call to the generator only. 2) A matrix of similar shape initialized to all values being one. These are our corresponding squared gradients.

### c) Compute loss (1.5)
Write a function `compute_loss(v1, v2, b1, b2, cooc_score, max_score, alpha)` that computes the GloVe Loss for a single pair of vectors v1 and v2 their weights b1 and b2 with associated cooc_score and weighting parameters max_score and alpha. The function returns 1) the loss and 2) the part $g$ that is the part of the gradient, that is the same in both

The GloVe Loss for a pair of indices i,j is: $loss(i,j) = f(C_{i,j}) \cdot \left(v_i \cdot v_j + b_i + b_j -log(C_{i,j})\right)^2$ with $C_{i,j}$ the cooc-score. The function function $f$ is defined as

$f(x) = (\frac{x}{max\_score})^{\alpha}$ if $x< max\_score$

$f(x) = 1 $ otherwise

The shared part is: $g(i,j) = f(C_{i,j}) \cdot \left(v_i \cdot v_j + b_i + b_j -log(C_{i,j})\right)$

### d) Computing the updates (0.5 each)

Write a function `calc_gradient_vi(g, vj, eta, grad_clip)`  that takes the value $g$ and a vector $v_j$, the learning rate $\eta$ and a gradient clipping value. It computes the gradient update for the $v_i$ vector without the bias. Applies gradient clipping such that the absolute value of each element of the gradient vector is at most grad_clip. Thereafter the gradient is multiplied with the learning rate and finally the validify function is applied to the gradient vector which is then returned.

Write a similar function to compute the update for the bias b `calc_gradient_b(g, grad_clip)`. It computes the gradient update for the bias b. By clipping the gradient and therafter validifying the gradient. It then returns the gradient update.
To compute the gradient update, drop the factor 2 that arises when differentiating the squared term. Also the gradient update for the bias terms does not include the learning rate.


### e) Apply the update (1.5)

Write a function `one_update(W, W_grad, pair, cooc_score, max_score, alpha, eta, grad_clip)`
That performs an update of the vector matrix W and gradient matrix W_grad for a pair of word indices $(i,j)$=pair which with associated cooc_score. The remaining parameters should be clear from the previous functions.

The matrix W is updated using the previously computed updates divided by the square root of the corresponding entries in the W_grad matrix. Keep in mind you have to walk into the opposite direction of the gradient. 

The W_grad matrix is updated using the sqared entry of the corresponding update (prior to dividing).

The function returns the loss prior to upgrading. (The matrices W and W_grad are updated in place)

### f) Write a training function (1)

Write function `train(W, W_grad, cooc_tpl, n_epochs, rng, max_score, alpha, eta, grad_clip)` that trains W and W_grad matrix using the cooc_tpl for n_epochs. Before each epoch shuffle all arrays in the cocc_tpl in unison using one call to `rng.permutation`. 

### g) Bringing it all together (1)

Write a function `train_corpus` that takes a corpus in the form of a list of strings.
Preprocess each string as in task 2). Keep only words that have at least three occurences. 

Internally use: max_score=100, alpha=3/4, window size of 4, learning rate of 0.05, dimension of glove vectors = 50, grad_clip=100. Create a new `numpy.random.default_rng` instance, feed it a seed of 1. Train for 25 epochs.

The function returns the trained W matrix.

Apply it to your corpus that you obtained in task 1).

In [347]:
from numpy.random import default_rng
rng = default_rng()

In [354]:
def init_matrix(v, n, rng):
    glove_vectors = rng.uniform(-0.5/n, 0.5/n, size=(v,n+1))
    squared_gradients = np.ones(shape=(v,n+1))
    return glove_vectors, squared_gradients 

In [362]:
def compute_loss(v1, v2, b1, b2, cooc_score, max_score, alpha):
    def f(x):
        if x < max_score:
            return (x / max_score) ** alpha
        else:
            return 1
    
    m = (np.matmul(v1.T, v2) + b1 + b2 - np.log(cooc_score))
    
    loss = f(cooc_score) * m ** 2
    g = f(cooc_score) * m
    
    return loss, g

### d) Computing the updates (0.5 each)

Write a function `calc_gradient_vi(g, vj, eta, grad_clip)`  that takes the value $g$ and a vector $v_j$, the learning rate $\eta$ and a gradient clipping value. It computes the gradient update for the $v_i$ vector without the bias. Applies gradient clipping such that the absolute value of each element of the gradient vector is at most grad_clip. Thereafter the gradient is multiplied with the learning rate and finally the validify function is applied to the gradient vector which is then returned.

Write a similar function to compute the update for the bias b `calc_gradient_b(g, grad_clip)`. It computes the gradient update for the bias b. By clipping the gradient and therafter validifying the gradient. It then returns the gradient update.
To compute the gradient update, drop the factor 2 that arises when differentiating the squared term. Also the gradient update for the bias terms does not include the learning rate.

In [367]:
def validify(grad):
    if np.isnan(grad).sum()>0 or np.isinf(grad).sum()>0:
        print('Warning: invalid value')
        return np.zeros(np.shape(grad))
    return grad



def calc_gradient_vi(fdiff, v, eta, grad_clip):
    grad = fdiff * v
    grad = np.clip(grad, -grad_clip, grad_clip)
    grad *= eta
    
    validify(grad)
    return grad
    

def calc_gradient_b(fdiff, grad_clip):
    pass

def one_update(W, W_grad, pair, cooc_score, max_score, alpha, eta, grad_clip):
    pass

def init_matrices(v, n, rng):
    glove_vectors = rng.uniform(-0.5/n, 0.5/n, size=(v,n+1))
    squared_gradients = np.ones(shape=(v,n+1))
    return glove_vectors, squared_gradients 

In [10]:
def train(W, W_grad, cooc_tpl, n_epochs, rng, max_score, alpha, eta, grad_clip):
    pass

In [11]:
# Small example of how it works:
from numpy.random import default_rng

# cooc_stuff
#cooc_dict, vocab = get_cooc_matrix(mini_corpus, 2)
#cooc_tpl = cooc_to_numpy(cooc_dict)

#rng = default_rng(1)
#W, W_grad = init_matrices(len(vocab), 2, rng)


#print(W)
#train(W, W_grad, cooc_tpl, n_epochs=25, rng=rng, max_score=2, alpha=0.8, eta=0.1, grad_clip=1)
#print(W)
#print(W_grad)

In [12]:
# What I obtain for code above

# W init
#[[ 0.00591081  0.22523185 -0.17792019]
# [ 0.22432472 -0.09408427 -0.03833678]
# [ 0.1638513  -0.04540043  0.02479684]
# [-0.23622044  0.12675655  0.01907166]]


# after training:
#loss on last pass 0.7861722683546024
#[[ 0.37861754  0.05811192  0.13090998]
# [ 0.67430958 -0.00918572  0.58465184]
# [-0.4713854   0.08926127 -0.31768868]
# [-0.58445333  0.02673079 -0.0975816 ]]
#[[ 1.01016694  1.0008865  12.1206226 ]
# [ 1.003497    1.00143482  9.65947768]
# [ 1.00864141  1.0002788   9.17531255]
# [ 1.00211807  1.00053237  6.48303266]]